In [1]:
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
%matplotlib inline

Using TensorFlow backend.
/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tensorflow/l

In [2]:
filename = r'Womens Clothing E-Commerce Reviews.csv'
WomensClothing= pd.read_csv(filename,encoding='latin-1')
WomensClothing['Review Text'] = WomensClothing['Review Text'].astype(str)
train, test = train_test_split(WomensClothing, test_size=0.20, random_state=2019, stratify = WomensClothing[['Recommended IND']])
labels = train['Recommended IND']
str(train)

'       Clothing ID  Age                                              Title  \\\n18342         1092   43            Light weight, pockets, great for summer   \n512           1078   49  Cute but skip the petite unless you are very s...   \n8334          1047   38                                            Perfect   \n22294         1077   58                                           Love it!   \n2883           829   69                                            Love it   \n...            ...  ...                                                ...   \n5714           880   39                                                NaN   \n22500         1098   57                            Embroidered masterpiece   \n16279         1078   38                      Gorgeous dress - spectacular!   \n18535           24   29                                 Super unflattering   \n17363          912   51                                        Lint magnet   \n\n                                             Rev

In [3]:
vocab_size = 10000
encoded_docs=[]
index = 1
for index, row in train.iterrows():
    if index % 10000 ==0:
        print(index)
    # take out the esentiment text
    text = row['Review Text']
    encoded_docs.append(one_hot(text, vocab_size))

20000


In [4]:
max_length = len(max(encoded_docs,key=len))
max_length
#make sure all the items have the uniform length
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[7953 4197  534 ...    0    0    0]
 [3421 2002 5064 ...    0    0    0]
 [7953 9119 4448 ...    0    0    0]
 ...
 [2735 8635 6907 ...    0    0    0]
 [1409 8101 3212 ...    0    0    0]
 [4747 6127 5164 ...    0    0    0]]


In [12]:
epoch=10
model = Sequential()
#specify the vocab size, dimensionality of the word embedding spaces
model.add(Embedding(vocab_size, 50, input_length=max_length))
#then we map the matrices to one-dimensional vector by calling flatten() funciton
model.add(Flatten())
#binary classifiction, 1 neuron, activation sigmoid; produce probablity
model.add(Dense(1, activation='sigmoid'))
# compile the model
#binary classifiction, loss = 'binary_crossentropy'
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()
# fit the model
#1st argument is the features/predictors/Xs; 2nd is the label/target/Ys
#ecpochs: pass the whole data to the Neural net work for 50 times
#verbose = 0; don't print anything
#verbose = 1; print key outputs
#vervose = 2; print all the info 
model.fit(padded_docs, labels, epochs=epoch, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 116, 50)           500000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 5800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5801      
Total params: 505,801
Trainable params: 505,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
 - 4s - loss: 0.3536 - acc: 0.8520
Epoch 2/10
 - 4s - loss: 0.2207 - acc: 0.9098
Epoch 3/10
 - 4s - loss: 0.1609 - acc: 0.9403
Epoch 4/10
 - 4s - loss: 0.1067 - acc: 0.9682
Epoch 5/10
 - 4s - loss: 0.0670 - acc: 0.9842
Epoch 6/10
 - 4s - loss: 0.0441 - acc: 0.9916
Epoch 7/10
 - 4s - loss: 0.0313 - acc: 0.9944
Epoch 8/10
 - 4s - loss: 0.0239 - acc: 0.9956
Epoch 9/10
 - 4s - loss: 0.0192 

In [13]:
encoded_docs_test=[]

for index, row in test.iterrows():
    if index % 10000 ==0:
        print(index)
    text = row['Review Text']

    encoded_docs_test.append(one_hot(text, vocab_size))

0
10000


In [23]:
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
print(padded_docs_test)

labels_test = test['Recommended IND']
loss, accuracy = model.evaluate(padded_docs_test, labels_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

[[2735 6907 2673 ...    0    0    0]
 [ 421 4475 8879 ...    0    0    0]
 [7325 2735 8635 ...    0    0    0]
 ...
 [7953 9119 2735 ...    0    0    0]
 [1344 6867 7953 ...    0    0    0]
 [7953 7574 8879 ...    0    0    0]]
Accuracy: 89.016603
